# Simplicial Complex Analysis

<span style='color:Red'>TODO: Get a system to determine ecological state of a shape</span> <br>
<span style='color:Red'> Currently using the largest number of nodes in each shape. </span> <br> 
Written by Frederick Miller, Casey McKean, and Wako Bungula. <br> 
The kepler mapper object gives an output that is not easily navigatible. To resolve this, we wish to create shapes that are easier to navigate and understand, and reveal the data inside of them. <br>
We generate all the shapes in the simplicial complex, condense 1-simplices where possible, and obtain summary statistics on the shapes and the nodes within the shapes.

In [1]:
import numpy as np
import pandas as pd 
import queue
import animation
import json
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
import pickle
print("Imports Done")

Imports Done


# File paths, `.p`, and `.json` import

From the `kmapper_demo` file, I added one extra code block to place the resulting simplicial complices in a `.json` file, which is a way to store dictionaries in long term storage. Additionally, it stores the dictionary of dataframes in a `.p` file, which is similar. The code below only needs to have the file paths changed, and then it will read the simplicial complices generated from kepler mapper. <br>
Here, we also import the actual data set, with data interpolated for the specific pool. <br>
Lastly, a list of the 11 continuous variables (the interpolated versions) is created.

In [2]:
jsonFilePath = r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\TDA1PCA10Cubes30perc_complices.json"
jsonFile = open(jsonFilePath, "r")
jsonData = json.load(jsonFile) 
jsonFile.close()

dataFilePath = r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\TDA1PCA10Cubes30perc_dfs.p"
df_dict = pickle.load(open(dataFilePath, "rb"))

variables = ["PredictedWDP", "PredictedSECCHI", "PredictedTEMP", "PredictedDO", 
           "PredictedTURB", "PredictedVEL", "PredictedTP", #"PREDICTED_COND",
           "PredictedTN", "PredictedSS", "PredictedCHLcal"]

variables = ["PREDICTED_WDP", "PREDICTED_SECCHI", "PREDICTED_TEMP", "PREDICTED_DO", 
           "PREDICTED_TURB", "PREDICTED_VEL", "PREDICTED_TP", #"PREDICTED_COND",
           "PREDICTED_TN", "PREDICTED_SS", "PREDICTED_CHLcal"]



print("Json file imported")

print(len(jsonData.keys()))

Json file imported
25


# Functions
See the `docstring`'s for what each function does and how it works.

In [43]:
def getSubdf(scomplex, shape, df):
    """
    Returns the part of the data frame from the particular shape in the simplicial complex.
    params:
    scomplex: the entire simplicial complex
    shape: the particular shape being inspected (within the simplicial complex)
    df: the entire data frame
    
    Description:
    1. Get all the nodes from the particular simplicial complex. 
    2. Generate the indices we care about from the particular shape. To do this, we read each node and append it's 
    indices to a list. Then, we convert the list to a set and then back to a list to eliminate duplicates.
    3. Return the dataframe with only those indices.
    """
    nodes = scomplex.get('nodes')
    indices = []
    npShape = np.array(shape).flatten()
    for node in npShape:
        indices.append(nodes.get(node))
    indices = list(set([item for sublist in indices for item in sublist]))
    subdf = df.loc[indices]
    return subdf

def shapeDataSummary(scomplex, shape, df, variables, verbose = False):
    """
    Generates summary statistics of the given variables for a given shape in the simplicial.
    params:
    scomplex: the entire simplicial complex
    shape: the particular shape being inspected (within the simplicial complex) at this function call.
    df: the entire dataframe
    variables: the variables of interest
    verbose: Determines if the function will print out extra information. False by default
    
    Description:
    1. Create an empty result dataframe to store the summary statistics.
    2. Get the sub dataframe (see getSubdf) for the particular shape
    3. For each variable we are analzying, generate summary statistics from the sub dataframe and place them
    inside the result dataframe.
    4. Return the result dataframe
    
    NOTE: this only creates summaries for one particular shape. In executing this method, it is done for each shape 
    outside of the function.
    
    """
    result = pd.DataFrame()
    if verbose == True:
        print("Obtaining sub dataframe for: ", shape)
        print("The number of nodes in this shape is: ", len(shape))
    subdf = getSubdf(scomplex, shape, df)
    if verbose == True:
        print("The number of datapoints in this shape is: ", subdf.shape[0])
    for var in variables:
        result[var] = subdf[var].describe()
    return result
    
    

def adjacent(v, scomplex):
    """
    Determines the nodes adjacent to a given vertex
    
    params:
    v: vertex
    scomlex: the entire simplicial complex
    
    Description:
    Determines the nodes that are adjacent to a given vertex.
    """
    
    simplices = scomplex.get('simplices')
    edges = [item for item in simplices if len(item) == 2]
    result = []
    for edge in edges:
        if v in edge:
            for item in edge:
                if item != v:
                    result.append(item)
    return result

def bfs(node, scomplex):
    """
    Conducts a breadth first search to obtain the entire shape from a given node
    params:
    node: the start node
    scomplex: the entire simplicial complex
    
    Description:
    Preforms a breadth first search to obtain the entire shape for a given start node.
    """
    Q = queue.Queue()
    result = []
    result.append(node)
    Q.put(node)
    while not Q.empty():
        v = Q.get()
        adjacentEdges = adjacent(v, scomplex)
        for edge in adjacentEdges:
            if edge not in result:
                result.append(edge)
                Q.put(edge)
    return result


        
    
def getShapes(scomplex):
    """
    Gets all of the shapes from a given simplicial complex.
    
    params:
    scomplex: the entire simplicial complex
    
    Description:
    1. Obtain all the nodes for the entire complex
    2. For each node, preform a breadth first search to obtain everything in that particular shape. 
    If this entire shape has not already been discovered, add it to the set of results. 
    The result item is a set as the order of the shapes does not matter. The resulting shape is a frozenset
    which means items cannot be added or removed once created, and is needed to allow the set object to have other sets within it.
    3. Convert each shape to a list and the result to a list for easier navigation outside of the function.
    4. Return the result
    
    """
    
    nodes = list(scomplex.get('nodes').keys())
    result = set()
    for node in nodes: # currently does more computations than necessary due to going through every node without considering it is already in a shape
        bfsResult = frozenset(bfs(node, scomplex))
        result.add(bfsResult)
    result = [list(x) for x in result]
    # Sort the list depending on what is decided: nodes or indices. Currently doing it by number of nodes
    result.sort(key = len, reverse = True)
    
    
    
    return result

def nodeDataSummary(node, scomplex, variables,df):
    """
    Returns a data summary of a particular node
    params:
    node: node in question
    scomplex: The entire simplicial complex
    variables: The variables to obtain summaries
    df: the entire dataframe 
    
    description:
    1. Creates a result dataframe
    2. Get all the indices from the node from the simplicial complex
    3. Generate summaries for each variable
    4. Return the result
    """
    result = pd.DataFrame()
    if isinstance(node, list):
        print("Node is a list and thus cannot get summary information: ", node)
        indices = scomplex.get('nodes').get(node[0])
        subdf = df.loc[indices]
        for var in variables:
            result[var] = subdf[var].describe()
        return result
    
        
    indices = scomplex.get('nodes').get(node)
    subdf = df.loc[indices]
    for var in variables:
        result[var] = subdf[var].describe()
    return result
    
    
def condenseShape(shape, scomplex):
    """
    
    params:
    shape: a shape of two nodes. must be 2
    scomplex: the entire simplicial complex
    
    description:
    gets the two nodes a and b
    gets the indices for a and b (what is inside the nodes)
    if a \subseteq b, return b
    elif b \subseteq a, return a 
    else return shape 
    
    """
    nodes = scomplex.get('nodes')
    a = shape[0]
    b = shape[1]
    aIndices = set(nodes.get(a))
    bIndices = set(nodes.get(b))
    
    if aIndices.issubset(bIndices):
        return b
    elif bIndices.issubset(aIndices):
        return a
    else:
        return shape

def clean_getShapes(scomplex):
    """
    Condenses 1-simplices down to 0-simplices when each node 
    is a subset of the other 
    
    params:
    scomplex: the entire simplicial complex
    
    Description:
    1. Get all the shapes from the original getShapes function
    2. For shapes that of length 2, if one is a subset of the other, return the larger of the two
        Otherwise, do nothing
    3. return the clean Shapes list 
    
    """
    shapes = getShapes(scomplex)
    cleanShapes = []
    for shape in shapes:
        if len(shape) == 2:
            shape = condenseShape(shape, scomplex)
            cleanShapes.append([shape])
        else:
            cleanShapes.append(shape)
    return cleanShapes


def getBoxplots(subdf, shape, key,filePath):
    """
    Generates box plots for 10 of the 11 continuous variables
    NOTE: CONDUCTIVITY IS NOT INCLUDED
    
    params:
    subdf: the sub dataframe of the particular shape
    shape: the shape in question
    key: what strata year season combo we are looking at 
    filePath: the output file path for all the box plots 
    
    description:
    clears the current plot 
    generates the sub dataframes for the respective variables.
    the reason they are grouped is based upon the numerical outputs for making the boxplots readable
    create a box plot, and then save it based upon the file path
    clear the plot
    repeat for the second set of variables
    """
    plt.clf()
    varDf1 = subdf[["PREDICTED_SS","PREDICTED_TURB","PREDICTED_CHLcal"]]
    varDf2 = subdf[["PREDICTED_TP","PREDICTED_TN","PREDICTED_TEMP","PREDICTED_DO","PREDICTED_VEL","PREDICTED_WDP",
                  "PREDICTED_SECCHI"]]
    plot1 = varDf1.boxplot()
    plt.savefig(filePath + "\\" + key +"_" + str(shape)  + "_SS_TURB_CHLcal"  + ".png")
    plt.clf()
    plot2 = varDf2.boxplot(rot = 45)
    plt.savefig(filePath + "\\" + key +"_" + str(shape)  + "_" + "TP_TN_TEMP_etc" + ".png")
    return plot1, plot2

def determineOverlap(scomplex, shapes, verbose = True):
    """
    Determines the overlap within a shape. 
    
    For each node, find it's neighbors, and generate the intersection, and saving the result without
    duplicates through utlizing the set functionality of python.
    
    params:
    scomplex: the entire simplicial complex in question
    shapes: all the shapes
    verbose: see prinout as the code works
    """
    
    shape = shapes[0] # chosen arbitrarily 
    
    result = set()
    if verbose:
        print("Shape: ", shape)
    nodes = scomplex.get('nodes')
    for node in shape:
        # currently, this displays a lot of repeats. 
        A = set(nodes.get(node))
        if verbose:
            print("Node: " , node  , " | Indices: ",A)
        B = adjacent(node, scomplex)
        if verbose:
            print("Adjacent nodes: ",B)
        for b in B:
            bSet = set(nodes.get(b))
            name = str(node) + " -> " + str(b) +": "
            intersection = set(A.intersection(bSet))
            intersection.add(name)
            intersection = frozenset(intersection)
            result.add(intersection)
            if verbose:
                print("Node: ", b, " | Indices: ", bSet)
                print("Overlap is: ", A.intersection(bSet))
    return result
print("Functions loaded")

Functions loaded


# Generating Summary Statistics on the entire simplicial complex
For each `mapper` output from `kepler-mapper`, we can generate the summary statistics for each of the continuous variables. This is done by first obtaining a list of the keys from the `.json` file, and then iterating through each complex, generating the shape and obtaining data summaries on each shape.

In [4]:
allComplices = list(jsonData.keys())
for key in allComplices: # remove indices here to get all the strata for all the time periods
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    shapes = clean_getShapes(scomplex)
    for shape in shapes:
        summaries = shapeDataSummary(scomplex, shape, df_dict.get(key), variables, verbose = False)
        if summaries.loc['count'][0] > 5 and len(shape)  > 2: # at least 6 datapoints and 3 nodes to see info
            print("The shape is: ",shape)
            print("The number of nodes in the shape is: ", len(shape))
            display(summaries) # Uncomment to see summaries

Current Simplical Complex:  ['Stratum 1 SUMMER 93-00: ']
The shape is:  ['cube6_cluster1', 'cube0_cluster0', 'cube2_cluster0', 'cube5_cluster0', 'cube8_cluster1', 'cube1_cluster0', 'cube4_cluster0', 'cube7_cluster1', 'cube5_cluster2', 'cube3_cluster0']
The number of nodes in the shape is:  10


,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000
mean,5.807438,39.445314,28.055503,7.612425,31.490741,0.311708,0.173857,3.620438,46.316052,25.681036
std,2.689415,10.622191,1.600216,1.281878,13.943474,0.248585,0.064355,0.637487,21.792598,10.487500
min,0.340000,14.000000,25.800000,5.400000,11.000000,0.000000,0.033480,2.432000,12.700000,9.751950
25%,4.000000,32.500000,26.675000,6.875000,22.000000,0.019134,0.147750,3.282750,31.700000,18.313660
50%,5.350000,40.000000,27.900000,7.500000,27.500000,0.332943,0.172609,3.664633,41.850000,23.866420
75%,7.600000,46.000000,29.200000,8.025000,36.500000,0.493413,0.206250,4.039438,59.900000,29.584580
max,12.200000,68.000000,32.900000,13.200000,88.000000,0.971381,0.380512,5.372994,108.300000,60.993300


Current Simplical Complex:  ['Stratum 2 SUMMER 93-00: ']
The shape is:  ['cube6_cluster1', 'cube0_cluster0', 'cube2_cluster0', 'cube8_cluster0', 'cube5_cluster0', 'cube8_cluster1', 'cube7_cluster0', 'cube1_cluster0', 'cube4_cluster0', 'cube9_cluster1', 'cube4_cluster2', 'cube3_cluster0', 'cube2_cluster3']
The number of nodes in the shape is:  13


,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000
mean,3.770625,35.100000,27.993179,7.773181,37.765000,0.544608,0.185213,3.636574,57.774063,26.393068
std,1.969209,10.464445,1.389663,1.491258,18.169783,0.264792,0.067865,0.656414,31.614296,12.546677
min,0.550000,12.000000,25.581729,4.400000,11.000000,0.000000,0.041000,2.255000,14.300000,5.911680
25%,2.100000,26.750000,26.800000,6.800000,25.000000,0.377500,0.153345,3.259775,33.900000,15.399952
50%,3.500000,35.000000,27.800000,7.600000,34.000000,0.520000,0.176550,3.612500,48.923178,24.135400
75%,5.000000,43.000000,29.200000,8.100000,46.000000,0.700000,0.211986,4.128250,69.950000,34.470165
max,11.300000,63.000000,31.600000,14.300000,99.000000,1.700000,0.723927,5.849319,151.400000,65.769350


Current Simplical Complex:  ['Stratum 3 SUMMER 93-00: ']
The shape is:  ['cube0_cluster0', 'cube2_cluster15', 'cube1_cluster0', 'cube2_cluster10', 'cube2_cluster18', 'cube2_cluster1']
The number of nodes in the shape is:  6


...

,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000
mean,0.919173,22.756958,28.723308,8.717293,52.315789,0.029850,0.317627,2.247321,52.580150,53.671621
std,0.505967,8.682136,2.132394,2.981090,30.324100,0.117827,0.116237,0.932839,29.353381,26.629376
min,0.300000,9.000000,23.100000,1.100000,5.000000,0.000000,0.138000,0.930000,7.640000,12.912610
25%,0.550000,15.000000,27.800000,6.600000,28.000000,0.000000,0.247752,1.585523,30.680000,30.735000
50%,0.800000,22.000000,28.900000,8.400000,41.000000,0.000000,0.289822,1.987938,44.140000,51.081280
75%,1.200000,30.000000,30.100000,10.600000,75.000000,0.000000,0.366000,2.941437,79.150000,70.535380
max,3.000000,47.000000,34.300000,15.500000,116.000000,0.620000,0.959000,5.341000,125.630000,121.777920


Current Simplical Complex:  ['Stratum 4 SUMMER 14-20: ']
The shape is:  ['cube1_cluster0', 'cube0_cluster0', 'cube2_cluster0']
The number of nodes in the shape is:  3


,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.0,12.000000,12.000000,12.000000,12.000000
mean,1.258333,24.416667,26.991667,8.675000,32.833333,0.0,0.296585,1.387400,34.786073,53.868560
std,0.339942,5.775471,0.782140,1.649862,9.675023,0.0,0.042173,0.108285,6.148955,7.851244
min,0.850000,16.000000,25.900000,6.800000,21.000000,0.0,0.240000,1.262000,23.990000,43.351450
25%,0.995000,20.000000,26.375000,7.450000,24.750000,0.0,0.243549,1.295844,30.550000,47.457445
50%,1.150000,25.000000,26.800000,8.150000,33.000000,0.0,0.312067,1.354604,36.411440,51.779545
75%,1.535000,27.000000,27.550000,10.025000,38.750000,0.0,0.330938,1.485797,39.765000,60.964008
max,1.930000,35.000000,28.300000,12.000000,50.000000,0.0,0.345000,1.568000,42.220000,66.258580


Current Simplical Complex:  ['Stratum 5 SUMMER 14-20: ']
The shape is:  ['cube0_cluster0', 'cube2_cluster0', 'cube0_cluster2', 'cube5_cluster0', 'cube6_cluster0', 'cube1_cluster0', 'cube4_cluster0', 'cube3_cluster0']
The number of nodes in the shape is:  8


,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,1.958842,28.452632,29.240000,12.674737,30.978947,0.017474,0.240889,2.421064,33.654211,71.726039
std,0.867914,4.081011,1.478239,4.508927,8.772514,0.074562,0.095062,0.563783,10.997084,31.960131
min,0.260000,20.000000,26.500000,3.400000,13.000000,0.000000,0.139000,1.438000,15.040000,17.851060
25%,1.300000,26.000000,28.150000,9.200000,25.000000,0.000000,0.185765,2.107062,27.715000,44.177800
50%,2.000000,28.000000,29.200000,13.300000,30.000000,0.000000,0.201172,2.430420,31.220000,73.999150
75%,2.600000,30.000000,30.600000,15.700000,35.500000,0.000000,0.258601,2.760124,36.300000,94.769645
max,4.200000,42.000000,31.900000,22.900000,56.000000,0.470000,0.511000,3.458000,74.530000,152.640790


# Analyzing the largest structure per simplicial complex
Largest = Node count of the shape. The largest structure is likely to be the dominant feature of the stratum during this particular time period. As such, it is important to analyze the nodes within it. To do this, we generate all the shapes, and since the shapes are returned in descending order of the number of nodes per shape, we pull the first shape. From here, we can preform an analysis on each one.

In [44]:
allComplices = list(jsonData.keys())

for key in allComplices: # remove the indices here to get all the strata for all the time periods
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    largestShape = clean_getShapes(scomplex)[0]
    nodes = scomplex.get('nodes')
    print("Largest shape is: ", largestShape)
    print("Number of nodes is: ", len(largestShape))
    for node in largestShape:
        summary = nodeDataSummary(node, scomplex,variables,df_dict.get(key))
        if summary.loc['count'][0] > 5: # 5 is chosen arbitraily
            print("Information for: ", node)
            display(summary)

Current Simplical Complex:  ['Stratum 1 SUMMER 93-00: ']
Largest shape is:  ['cube4_cluster0', 'cube5_cluster0', 'cube0_cluster0', 'cube1_cluster0', 'cube6_cluster1', 'cube5_cluster2', 'cube2_cluster0', 'cube7_cluster1', 'cube8_cluster1', 'cube3_cluster0']
Number of nodes is:  10
Information for:  cube4_cluster0


,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,5.927500,33.500000,27.814025,7.604477,38.200000,0.298900,0.189691,3.630190,62.784683,26.714508
std,3.133498,6.778449,1.802823,0.959260,7.911351,0.283599,0.061191,0.487281,9.150042,6.916225
min,0.350000,23.000000,25.900000,6.100000,21.000000,0.000000,0.058000,2.637000,49.874813,16.934920
25%,4.000000,28.000000,26.500000,7.000000,33.750000,0.015965,0.171182,3.473000,58.700000,22.030800
50%,5.400000,33.000000,27.200000,7.650000,39.000000,0.215274,0.180000,3.692766,61.850000,24.433895
75%,8.625000,39.000000,28.600000,8.050000,44.250000,0.506116,0.206250,3.860921,64.239005,33.219975
max,12.200000,46.000000,32.900000,10.100000,50.000000,0.971381,0.365100,4.642000,88.858608,42.224000


Information for:  cube5_cluster0


,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
mean,3.807143,26.142857,28.985714,7.871429,49.142857,0.438676,0.158788,3.873822,72.414286,28.956149
std,1.778208,3.184785,2.741915,1.542416,5.080307,0.267538,0.070747,0.674105,7.990709,13.200369
min,0.350000,23.000000,26.000000,5.400000,41.000000,0.000000,0.058000,2.637000,62.000000,12.129260
25%,3.500000,23.500000,26.650000,7.150000,47.000000,0.303211,0.109008,3.651500,66.000000,20.209190
50%,4.000000,25.000000,28.400000,7.800000,50.000000,0.465320,0.187000,3.966000,73.400000,25.223080
75%,4.700000,28.500000,31.150000,8.750000,51.000000,0.656281,0.206750,4.284376,79.000000,38.391130
max,5.900000,31.000000,32.900000,10.100000,57.000000,0.686430,0.235000,4.642000,81.500000,48.140060


Information for:  cube0_cluster0


...

,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,1.683214,27.000000,29.010714,7.646429,37.178571,0.043214,0.217565,2.375582,39.930357,37.617895
std,0.830894,3.953433,1.275507,2.130638,7.463757,0.119196,0.059569,0.524943,11.603750,9.513446
min,0.620000,20.000000,26.700000,3.400000,21.000000,0.000000,0.139000,1.464000,21.600000,17.851060
25%,0.945000,24.000000,28.275000,6.450000,32.500000,0.000000,0.187387,2.107093,30.012500,32.618400
50%,1.550000,27.000000,29.200000,8.150000,37.000000,0.000000,0.202037,2.403506,37.585000,39.902885
75%,2.150000,29.250000,29.725000,9.350000,41.250000,0.000000,0.213250,2.758986,46.917500,44.107650
max,3.400000,36.000000,31.600000,10.600000,52.000000,0.470000,0.420212,3.279035,62.890000,56.337990


Information for:  cube2_cluster0


,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,2.013077,28.961538,29.461538,11.146154,28.192308,0.021154,0.214269,2.338609,29.769231,56.395755
std,0.957876,4.787323,1.638311,3.512860,8.777331,0.067547,0.074985,0.505168,9.105724,9.571153
min,0.260000,22.000000,26.500000,4.000000,13.000000,0.000000,0.139000,1.464000,15.040000,40.470050
25%,1.400000,26.000000,28.675000,9.500000,21.250000,0.000000,0.171882,2.118810,22.342500,49.580780
50%,1.900000,29.000000,29.850000,10.900000,27.500000,0.000000,0.193077,2.293081,28.860000,56.823795
75%,2.600000,30.000000,30.875000,13.375000,34.750000,0.000000,0.212213,2.685055,35.200000,64.362060
max,4.200000,42.000000,31.700000,18.000000,44.000000,0.310000,0.444069,3.420048,53.190000,76.985400


Information for:  cube3_cluster0


,PREDICTED_WDP,PREDICTED_SECCHI,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_VEL,PREDICTED_TP,PREDICTED_TN,PREDICTED_SS,PREDICTED_CHLcal
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.0,37.000000,37.000000,37.000000,37.000000
mean,2.080541,29.054054,29.335135,15.097297,27.351351,0.0,0.240078,2.364704,29.151081,86.741724
std,0.730608,3.843797,1.569716,2.569964,5.458002,0.0,0.095537,0.616952,4.475235,10.615325
min,0.500000,22.000000,27.100000,10.700000,16.000000,0.0,0.147000,1.438000,18.540000,63.289500
25%,1.800000,27.000000,28.100000,13.300000,25.000000,0.0,0.184500,1.900864,26.780000,81.910500
50%,2.300000,28.000000,28.600000,14.700000,27.000000,0.0,0.198950,2.314600,29.600000,87.216900
75%,2.600000,30.000000,30.900000,16.500000,31.000000,0.0,0.256202,2.653500,31.710000,94.482790
max,3.300000,37.000000,31.800000,21.100000,38.000000,0.0,0.511000,3.458000,37.320000,104.232810


# Condensing 1-simplices
Currently, many one simplices that we have contain information that means one of them is a subset of the other. To resolve this, we replace them with one cluster with all the indices in one node.

This is stored in the function `clean_getShapes(scomplex)` function. Below is a comparison of running the two functions

In [ ]:
allComplices = list(jsonData.keys())
print("Standard shape version")
for key in allComplices[0:1]:
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    nodes = scomplex.get('nodes')
    shapes = getShapes(scomplex)
    for shape in shapes:
        indices = []
        for node in shape:
            indices.append(nodes.get(node))
        indices = list(set([item for sublist in indices for item in sublist]))
        print(str(shape) + " : " + str(indices))

print("Clean shape version")
for key in allComplices[0:1]:
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    nodes = scomplex.get('nodes')
    cleanShapes = clean_getShapes(scomplex)
    for shape in cleanShapes:
        indices = []
        npShape = np.array(shape).flatten()
        for node in npShape:
            indices.append(nodes.get(node))
        indices = list(set([item for sublist in indices for item in sublist]))
        print(str(shape) + " : " + str(indices))

# Box plot per shape
Here, we generate box plots for the variables of interest. for each shape in the simplicial complex <br>
TODO: Plot output ideas: <br>
SS, Turb, CHLCal <br>
Vel, TN, TP <br>
Temp, DO, SECCHI <br>

In [ ]:
allComplices = list(jsonData.keys())
for key in allComplices[0:1]: # remove the indices here to get all the strata for all the time periods
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    shapes = clean_getShapes(scomplex)
    print("number of shapes: ", len(shapes))
    for shape in shapes:
        """
        Hello whoever is using this function
        getBoxplots takes a couple arguments. 
        the big thing that matters here is the 
        strataYear variable. Essentially, the key that allows us to access each simplicial complex is weird.
        Depending on your file system, using the str(key) conversion may cause errors. to resolve this, 
        below is a potential example. Feel free to change it as you go for your use.
        """
        strataYear = str(key).replace(" ","-").replace(":","").replace("[","").replace("]","").replace("'","")
        print(strataYear)
        """
        before: 
        ['Stratum 1 SUMMER 93-00: ']
        after:
        Stratum-1-SUMMER-93-00-
        """
        subdf = getSubdf(scomplex, shape, df_dict.get(key))
        plots = getBoxplots(subdf, shape, strataYear,  
                            filePath = r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\Boxplots")


# Discovering which indices within the nodes overlap
This code will determine what points in the nodes are overlapping within two nodes.

In [23]:
allComplices = list(jsonData.keys())
for key in allComplices[0:1]: # remove the indices here to get all the strata for all the time periods
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    shapes = clean_getShapes(scomplex)
    print("number of shapes: ", len(shapes))
    result = determineOverlap(scomplex, shapes, verbose = False)
    print(result)

Current Simplical Complex:  ['Stratum 1 SUMMER 93-00: ']
number of shapes:  8
{frozenset({97, 66, 'cube6_cluster1 -> cube5_cluster0: '}), frozenset({'cube2_cluster0 -> cube3_cluster0: ', 51, 22, 87, 72, 56, 76}), frozenset({59, 'cube4_cluster0 -> cube5_cluster0: ', 83, 4}), frozenset({98, 'cube7_cluster1 -> cube8_cluster1: '}), frozenset({67, 44, 'cube1_cluster0 -> cube0_cluster0: '}), frozenset({32, 1, 34, 65, 36, 69, 71, 'cube3_cluster0 -> cube4_cluster0: ', 62, 63}), frozenset({'cube7_cluster1 -> cube6_cluster1: ', 102}), frozenset({20, 'cube5_cluster2 -> cube4_cluster0: '}), frozenset({'cube0_cluster0 -> cube1_cluster0: ', 67, 44}), frozenset({0, 2, 5, 10, 11, 15, 18, 24, 26, 27, 28, 29, 43, 47, 111, 112, 114, 53, 54, 'cube1_cluster0 -> cube2_cluster0: '}), frozenset({0, 2, 5, 10, 11, 15, 18, 24, 26, 27, 28, 29, 43, 47, 111, 112, 114, 'cube2_cluster0 -> cube1_cluster0: ', 53, 54}), frozenset({98, 'cube8_cluster1 -> cube7_cluster1: '}), frozenset({'cube4_cluster0 -> cube5_cluster2: 

# See compare shapes over the years
NOTE: Comparing the largest shape in each.